In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import FloatVGG16
import SampleVGG16
from SampleNN import *
%matplotlib inline
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import torchvision

In [2]:
import torchvision.transforms as transforms
traindir = '/home/data/yanzy/train'
valdir = '/home/data/yanzy/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_dataset =  torchvision.datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                         shuffle=False, num_workers = 16, pin_memory=False)

train_dataset =  torchvision.datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,
                                         shuffle=True, num_workers = 4, pin_memory=False)

In [3]:
sample_net = SampleVGG16.SampleVGG16().to(device)

32


NameError: name 'it' is not defined

In [5]:
criterion_s = nn.CrossEntropyLoss()
optimizer_s = optim.SGD(sample_net.parameters(), lr=1e-6, momentum=0.9)
#optimizer_s = optim.Adam(sample_net.parameters(), lr=1e-6, weight_decay=1e-6)
def train(epoch):
    sample_net.train()
    running_loss = 0.0
    count = 0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer_s.zero_grad()

        # forward + backward + optimize
        TMP = protectStateDict(sample_net)
        sampleStateDict(sample_net)
        outputs = sample_net(inputs)
        loss = criterion_s(outputs, labels)
        loss.backward()
        sample_net.load_state_dict(TMP)
        del TMP
        torch.cuda.empty_cache()
        optimizer_s.step()
        count += labels.size(0)

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:
            print running_loss * 1.0 / (i+1)
        if count > 20000:
            break
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 1000))
    return running_loss

In [6]:
torch.cuda.empty_cache()
train(1)

0.309199333191


RuntimeError: CUDA error: out of memory

In [ ]:
def val():
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            images = oSample()(images)
            outputs = sample_net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            count += labels.size(0)
            correct += (predicted == labels).sum().item()
            #print correct, total
            if count >= 10000:
                print 'alive'
                count = 0

    print('Accuracy of the network on the %d test images: %.3f %%' % (total,
        100.0 * correct / total))
    return 100.0 * correct / total

In [8]:
torch.cuda.empty_cache()
TMP = protectStateDict(sample_net)
sampleStateDict(sample_net)
val()
sample_net.load_state_dict(TMP)

alive
alive
alive
alive
Accuracy of the network on the 50000 test images: 63.280 %
